# Confidence Intervals Using Approximate Monte Carlo Simulation Iterations
## Conditionally Bias-adjusted RMSE-based Wald Confidence Intervals with Student’s T distribution
### Pandas

This notebook provides an example on how the PPMF files from the 25 Approximate Monte Carlo (AMC) iterations can be used to generate confidence intervals for a few example queries and geographies.

The method used is an RMSE-based confidence interval assuming a Student’s T distribution with 5 degrees of freedom. We use a conditional approach in
which a bias-corrected interval was used only if some criteria was met and otherwise the non-bias-corrected versions were utilized. 

Bias was estimated as the "true" count from the PPMF0 for a given query and geography subtracted from the average count from the 25 AMC iterations for the same query and geography.

For the bias-corrected interval, the estimated bias was subtracted from the point estimate of the confidence interval.
The non-bias-corrected version did not have an adjusted interval.

All three of the following criteria had to be met in order to use the bias-corrected version:
* The PPMF0 value was greater than 5
* The absolute value of the estimated bias divided by the estimated standard deviation of the AMC estimates was greater than or equal to 0.5
* The estimated bias was either negative or the PPMF0 value was at least 25

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

def ci_wald_t_conditional(df_data, ci_level = 0.9, degrees_freedom = 5):
    """ Calculate confidence intervals using conditionally bias-adjusted RMSE-based Wald CI with Student’s T distribution
        Note: as our query of interest is a count, we apply a floor and ceiling function to the endpoints of the confidence intervals respectively
        and truncate at zero to account for the outcome space being integer and non-negative
    :df_data: a pandas dataframe aggregated to the query x geography level which contains the following columns-
         "PPMF0_COUNT": the count from the PPMF0
         "MEAN": the average count from all simulation iterations
         "STD": the standard deviation of the counts from all the simulation iterations
         "MSE": the mean squared error, where error was calculated as the count from the PPMF0 subtracted from the count from a simulation run
    :ci_level: level for two-sided confidence interval (ex: .9)
    :deg_free: degrees of freedom for CI using Student's T distribution
    :return: the input pandas dataframe with a new column for the confidence interval
    """
    t_value = scipy.stats.t.ppf(q=1-(1-ci_level)/2, df=degrees_freedom) 
    print("Calculating confidence interval using t-value", t_value)
    df_data["RMSE"] = df_data["MSE"].pow(1/2)
    df_data["BIAS"] = df_data["MEAN"] - df_data["PPMF0_COUNT"]
    df_data["MET_CRITERIA"] = np.where((df_data["PPMF0_COUNT"] > 5) &
                                  (df_data["BIAS"].abs()/df_data["STD"] >= 0.5) &
                                  ((df_data["BIAS"] < 0) | (df_data["PPMF0_COUNT"] >= 25)),
                                  True, False)
    df_data["POINT_EST"] = np.where(df_data["MET_CRITERIA"], df_data["PPMF0_COUNT"] - df_data["BIAS"], df_data["PPMF0_COUNT"])
    df_data['CI'] = list(zip(np.floor((df_data["POINT_EST"] - t_value * df_data["RMSE"]).clip(lower=0)).astype(int), 
                    np.ceil(df_data["POINT_EST"] + t_value * df_data["RMSE"]).astype(int)))
    return df_data

For this example, we have already used the 26 PPMF files (the PPMF0 and the TDA outputs PPMF1 - PPMF25) to summarize counts 
for a few example queries and geographies. Note the full list of queries in the DHC and PL94 tables can be found here-

DHC: https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/complete-tech-docs/demographic-and-housing-characteristics-file-and-demographic-profile/2020-dhc-table-matrix.xlsx

PL: https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/complete-tech-docs/summary-file/2020Census_PL94_171Redistricting_StatesTechDoc_English.pdf


In [2]:
# Our data has one row for each query, geography, and iteration (0-25)

file_path = "jupyter_data.csv"
df_example = pd.read_csv(file_path)
df_example.head(10)

,PRODUCT_TABLE,QUERY,GEO,ITERATION,COUNT
0,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,0,1379
1,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1,1382
2,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,2,1375
3,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,3,1382
4,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,4,1373
5,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,5,1382
6,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,6,1389
7,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,7,1376
8,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,8,1380
9,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,9,1379


In [3]:
# Divide the csv datafile into two - the PPMF0 data (which was input to the TDA for the AMC method) 
# and the PPMF1-PPMF25 synthetic data (which was the output from the AMC method)

df_PPMF0 = df_example[df_example["ITERATION"] == 0]
df_PPMF1_25 = df_example[df_example["ITERATION"] != 0]

df_PPMF0 = df_PPMF0.rename(columns={"COUNT": "PPMF0_COUNT"})
df_PPMF0 = df_PPMF0.drop(columns=["ITERATION"])

# Then join the files back together to calculate the differences

df_calc = df_PPMF0.merge(df_PPMF1_25, on=["PRODUCT_TABLE", "QUERY", "GEO"], how="outer")
df_calc["DIFF"] = df_calc["COUNT"] - df_calc["PPMF0_COUNT"]
df_calc["ABS_DIFF"] = df_calc["DIFF"].abs()
df_calc["SQRD_ERROR"] = df_calc["DIFF"]**2

df_calc.head(10)

,PRODUCT_TABLE,QUERY,GEO,PPMF0_COUNT,ITERATION,COUNT,DIFF,ABS_DIFF,SQRD_ERROR
0,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,1,1382,3,3,9
1,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,2,1375,-4,4,16
2,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,3,1382,3,3,9
3,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,4,1373,-6,6,36
4,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,5,1382,3,3,9
5,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,6,1389,10,10,100
6,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,7,1376,-3,3,9
7,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,8,1380,1,1,1
8,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,9,1379,0,0,0
9,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,10,1376,-3,3,9


In [4]:
# For each query and unique geography, calculate the input statistics needed to construct the confidence intervals
# this requires calculating the mean and standard deviation from the 25 simulation iterations, and the mean squared error of the differences

# note this syntax only works for pandas >= 0.25
df_statistics = df_calc.groupby(["PRODUCT_TABLE", "QUERY", "GEO", "PPMF0_COUNT"],  as_index=False).agg(
    MEAN=("COUNT", "mean"),
    MSE=("SQRD_ERROR", "mean"),
    STD=("COUNT", "std"))


df_statistics.head(10)

,PRODUCT_TABLE,QUERY,GEO,PPMF0_COUNT,MEAN,MSE,STD
0,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,1379.68,14.92,3.880722
1,DHCH H4,H0040002: Owned mortgage,ST-CO-TR: 24-033-807200,106,106.28,5.40,2.354428
2,DHCH H4,H0040003: Owned no mortgage,ST-CO-TR-BG: 24-033-807200-1,17,14.20,20.08,3.570714
3,DHCH H4,H0040004: Rented,ST-CO-TR-BG: 24-033-807200-1,383,384.88,20.84,4.245782
4,DHCP PCT8,PCT0080001: Total NonVoting,STATE: 32,665025,665023.52,377.64,19.776080
5,DHCP PCT8,PCT0080002: NonVoting Household,STATE: 32,663773,663761.92,615.72,22.660391
6,DHCP PCT8,"PCT0080003: Nonvoting Householder, spouse",STATE: 32,744,795.76,3226.56,23.880396
7,DHCP PCT8,PCT0080021: Nonvoting Institutional,STATE-COUNTY: 32-021,13,11.72,32.08,5.631163
8,DHCP PCT8,PCT0080029: Nonvoting Non-Institutional,STATE-COUNTY: 32-021,0,0.00,0.00,0.000000
9,PL94 H1,H0010001: Total,ST-AIAN: 06-4255,1313,1313.00,0.00,0.000000


In [5]:
# Now calculate the 90% confidence interval using the aggregated pandas dataframe.
# We use 5 degrees of freedom, preferring a more conservative, larger interval

df_ci = ci_wald_t_conditional(df_data=df_statistics, ci_level=.9, degrees_freedom=5)
df_ci.head(15)

# Please note the following in our output:
# Query "PCT0080029: Nonvoting Non-Institutional" had a count of 0 from every AMC iteration and from the PPMF0. 
#    Therefore, the confidence interval is (0, 0)
# Query "H0010001: Total" measures the total number of housing units. This is an invariant, 
#    meaning the DAS TDA does not apply noise to this statistic. 
#    Therefore, every AMC simulation contains the same count and the confidence interval has a width of 0

Calculating confidence interval using t-value 2.015048372669157


,PRODUCT_TABLE,QUERY,GEO,PPMF0_COUNT,MEAN,MSE,STD,RMSE,BIAS,MET_CRITERIA,POINT_EST,CI
0,DHCH H4,H0040001: Total,ST-CO-TR: 24-033-807200,1379,1379.68,14.92,3.880722,3.862642,0.68,False,1379.00,"(1371, 1387)"
1,DHCH H4,H0040002: Owned mortgage,ST-CO-TR: 24-033-807200,106,106.28,5.40,2.354428,2.323790,0.28,False,106.00,"(101, 111)"
2,DHCH H4,H0040003: Owned no mortgage,ST-CO-TR-BG: 24-033-807200-1,17,14.20,20.08,3.570714,4.481071,-2.80,True,19.80,"(10, 29)"
3,DHCH H4,H0040004: Rented,ST-CO-TR-BG: 24-033-807200-1,383,384.88,20.84,4.245782,4.565085,1.88,False,383.00,"(373, 393)"
4,DHCP PCT8,PCT0080001: Total NonVoting,STATE: 32,665025,665023.52,377.64,19.776080,19.432962,-1.48,False,665025.00,"(664985, 665065)"
5,DHCP PCT8,PCT0080002: NonVoting Household,STATE: 32,663773,663761.92,615.72,22.660391,24.813706,-11.08,False,663773.00,"(663722, 663824)"
6,DHCP PCT8,"PCT0080003: Nonvoting Householder, spouse",STATE: 32,744,795.76,3226.56,23.880396,56.802817,51.76,True,692.24,"(577, 807)"
7,DHCP PCT8,PCT0080021: Nonvoting Institutional,STATE-COUNTY: 32-021,13,11.72,32.08,5.631163,5.663921,-1.28,False,13.00,"(1, 25)"
8,DHCP PCT8,PCT0080029: Nonvoting Non-Institutional,STATE-COUNTY: 32-021,0,0.00,0.00,0.000000,0.000000,0.00,False,0.00,"(0, 0)"
9,PL94 H1,H0010001: Total,ST-AIAN: 06-4255,1313,1313.00,0.00,0.000000,0.000000,0.00,False,1313.00,"(1313, 1313)"


### PySpark
Alternatively, when users are accessing all 26 of the full PPMF files, they may prefer to calculate the prior example using PySpark

In [6]:
import os, sys
import scipy.stats
from IPython.core.display import HTML

if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = '/usr/lib/spark'
    
sys.path.append(os.path.join(os.environ['SPARK_HOME'],'python'))
sys.path.append(os.path.join(os.environ['SPARK_HOME'],'python','lib', 'py4j-src.zip'))

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

def ci_wald_t_conditional_pyspark(df_data, ci_level = 0.9, degrees_freedom = 5):
    """ Calculate confidence intervals using conditionally bias-adjusted RMSE-based Wald CI with Student's T distribution
        Note: as our query of interest is a count, we apply a floor and ceiling function to the endpoints of the confidence intervals respectively
        and truncate at zero to account for the outcome space being integer and non-negative
    :df: a pyspark dataframe aggregated to the query x geography level which contains the following columns-
         "PPMF0_COUNT": the "true" count from the PPMF0
         "MEAN": the average count from all simulation runs
         "STD": the standard deviation of the counts from all the simulation runs
         "MSE": the mean squared error, where error was calculated as the count from the PPMF0 subtracted from the count from a simulation run
    :ci_level: level for two-sided confidence interval (ex: .9)
    :deg_free: degrees of freedom for CI using Student's T distribution
    :return: the input pyspark dataframe with a new column for the confidence interval
    """
    t_value = scipy.stats.t.ppf(q=1-(1-ci_level)/2, df=degrees_freedom) 
    print("Calculating confidence interval using t-value", t_value)
    df_data = df_data.withColumn("RMSE", F.sqrt(F.col("MSE")))\
                     .withColumn("BIAS", F.col("MEAN") - F.col("PPMF0_COUNT"))\
                     .withColumn("MET_CRITERIA", F.when((F.col("PPMF0_COUNT") > 5) &
                                              (F.abs(F.col("BIAS"))/F.col("STD") >= 0.5) &
                                              ((F.col("BIAS") < 0) | (F.col("PPMF0_COUNT") >= 25)),
                                              True).otherwise(False))\
                     .withColumn("POINT_EST", F.when(F.col("MET_CRITERIA"), F.col("PPMF0_COUNT") - F.col("BIAS"))
                                     .otherwise(F.col("PPMF0_COUNT")))\
                     .withColumn("CI", F.array(F.floor(F.greatest(F.lit(0), F.col("POINT_EST") - t_value * F.col("RMSE"))),
                                     F.ceil(F.col("POINT_EST") + t_value * F.col("RMSE"))))
    return df_data

In [ ]:
spark = SparkSession.builder.getOrCreate()

df_spark = spark.createDataFrame(df_example)

# We split the csv datafile into two - the PPMF0 "true" data and the PPMF1-PPMF25 synthetic data
df_PPMF0_spark = df_spark.filter("ITERATION = 0")
df_PPMF1_25_spark = df_spark.filter("ITERATION != 0")

df_PPMF0_spark = df_PPMF0_spark.withColumnRenamed("COUNT", "PPMF0_COUNT")
df_PPMF0_spark = df_PPMF0_spark.drop("ITERATION")

# Then join the files back together & calculate the differences
df_calc_spark = df_PPMF1_25_spark.join(df_PPMF0_spark, ["PRODUCT_TABLE", "QUERY", "GEO"], "outer")
df_calc_spark = df_calc_spark.withColumn("DIFF", F.col("COUNT") - F.col("PPMF0_COUNT"))
df_calc_spark = df_calc_spark.withColumn("ABS_DIFF", F.abs(F.col("DIFF")))
df_calc_spark = df_calc_spark.withColumn("SQRD_ERROR", F.pow(F.col("DIFF"), 2))

# For each query level and unique geography, calculate the statistics needed to construct the confidence intervals
# this includes calculating the mean, stdev, and the mean squared error of the differences
df_statistics_spark = df_calc_spark.groupBy(["PRODUCT_TABLE", "QUERY", "GEO", "PPMF0_COUNT"]).agg(
                    F.mean("COUNT").alias("MEAN"),
                    F.mean("SQRD_ERROR").alias("MSE"),
                    F.stddev("COUNT").alias("STD"))

df_ci_spark = ci_wald_t_conditional_pyspark(df_data=df_statistics_spark, ci_level=.9, degrees_freedom=5)

df_ci_spark.orderBy("PRODUCT_TABLE","QUERY").toPandas().head(15)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/28 15:05:31 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Calculating confidence interval using t-value 2.015048372669157
